#                                This program can support your investing decision

In [3]:
import pandas as pd
import numpy as np
import yfinance as yf

# 1. Correct the company's ticker you want to analyze 

In [4]:
# Correct top 500 U.S. comanies by market capitalization
df = pd.read_csv('data/nasdaq.csv')
df = df[df.Country == 'United States']
data = df.sort_values('Market Cap', ascending=False).head(500)
tickers = []
for _, i in data.iterrows():
    ticker = f"{i.Symbol}"
    tickers.append(ticker)

# 2. Find out undervalued stock by PER (Price / 3 years average of EPS)

In [ ]:
pers = []
for i, ticker in enumerate(tickers):
    symbol = yf.Ticker(ticker)

    try:
        # Initialze data
        ins = symbol.financials
        overview = symbol.info

        # Normal PER
        nper = overview.get('trailingPE')

        # Current price
        price = overview.get('regularMarketPrice', np.nan)

        # Get 3-year average of EPS
        flag = 0
        first = float(ins.loc['Diluted EPS'].iloc[0])
        if np.isnan(first):
            flag += 1
            first = 0
        second = float(ins.loc['Diluted EPS'].iloc[1])
        if np.isnan(second):
            flag += 1
            second = 0
        third = float(ins.loc['Diluted EPS'].iloc[2])
        if np.isnan(third):
            flag += 1
            third = 0
        forth = float(ins.loc['Diluted EPS'].iloc[3])
        if np.isnan(forth):
            flag += 1
            forth = 0
        avg = (first + second + third) / (4 - flag)

        # Graham PER
        gper = round(price / avg, 2)
        pers.append({'ticker': ticker, 'g_per': gper, 'n_per': nper, 'first': first, 'second': second, 'third': third, 'forth': forth})

    except Exception as e:
        print(f"Error processing {ticker}: {e}")
        continue 

df = pd.DataFrame(pers)
df = df[df.g_per > 0]
df.head()

Error processing BRK/A: HTTP Error 500: 
Error processing BRK/B: HTTP Error 500: 
Error processing CCZ: 'Diluted EPS'
Error processing TBB: 'Diluted EPS'
Error processing HBANL: 'Diluted EPS'
Error processing HBANM: 'Diluted EPS'
Error processing ECCF: 'Diluted EPS'
Error processing MCHPP: 'Diluted EPS'
Error processing HBANP: 'Diluted EPS'
Error processing CRWV: single positional indexer is out-of-bounds
Error processing STRF: 'Diluted EPS'


# Select companies have PER < 15¶

In [ ]:
selection = df[df.g_per < 15.0]

# Find out undervalued stock by PBR 

In [ ]:
undervalues = []
for _, i in selection.iterrows():
    symbol = yf.Ticker(i.ticker)

    try:
        # Initialze data
        overview = symbol.info
        per = i.g_per

        # Get PBR
        pbr = round(overview.get('priceToBook', np.nan), 2)

        # PER * PBR
        if pd.notna(pbr):
            per_pbr = round(per * pbr, 2)
        else:
            per_pbr = None

        undervalues.append({'ticker': i.ticker, 'PER': per, 'PBR': pbr, 'PER*PBR': per_pbr})

    except Exception as e:
        print(f"Error processing {i.ticker}: {e}")
        continue 

undervalue_df = pd.DataFrame(undervalues)

# Select companies have PBR < 1.5

In [ ]:
undervalue_df = undervalue_df[(undervalue_df['PBR'] < 1.5) & (undervalue_df['PBR'] > 0)]
undervalue_df